In [37]:
from WindPy import w
import pandas as pd
import numpy as np
import pyfolio as pf
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import sys
%matplotlib inline

In [38]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!
You can use w.menu to help yourself to create commands(WSD,WSS,WST,WSI,WSQ,...)!

COPYRIGHT (C) 2016 WIND HONGHUI INFORMATION & TECHKNOLEWDGE CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [39]:
data = w.wsd('040002.OF', "NAV_adj", "ED-%dD"%(0), '2017-04-18', "")

In [40]:
wind2df(data)

,nav_adj
0,4.352979


In [6]:
DATA_DIR = 'D:/Data/fund'
end_date = '2017-04-13'
rptDate = '20161231'

In [7]:
def wind2df(raw_data):
    dic = {}
    for data, field in zip(raw_data.Data, raw_data.Fields):
        dic[str(field.lower())] = data
    return pd.DataFrame(dic)

In [6]:
data = w.wset("sectorconstituent","date=2017-04-12;windcode=885012.WI")

In [7]:
df = wind2df(data)

In [9]:
fname = u'%s/股票型基金列表.xlsx'%(DATA_DIR)
df[['sec_name', 'wind_code']].to_excel(fname, index=False)

In [10]:
df = pd.read_excel(fname)

In [11]:
data = w.wss(df['wind_code'].tolist(), "fund_investtype")

In [12]:
temp = wind2df(data)

In [13]:
df['investtype'] = temp['fund_investtype']

In [14]:
df.head()

,sec_name,wind_code,investtype
0,嘉实中证500ETF联接,000008.OF,被动指数型基金
1,财通中证100增强,000042.OF,增强指数型基金
2,华夏沪深300ETF联接,000051.OF,被动指数型基金
3,国联安中证医药100,000059.OF,被动指数型基金
4,嘉实研究阿尔法,000082.OF,普通股票型基金


In [15]:
data = w.wss(df['wind_code'].tolist(), "issue_date")

In [16]:
temp = wind2df(data)

In [17]:
df['issue_date'] = temp['issue_date']

In [18]:
data = w.wss(df['wind_code'].tolist(), "prt_netasset","rptDate=%s"%(rptDate))

In [19]:
temp = wind2df(data)

In [20]:
df['netasset'] = temp['prt_netasset']

In [21]:
data = w.wss(df['wind_code'].tolist(), "fund_fundmanager")

In [22]:
temp = wind2df(data)

In [23]:
df['fundmanager'] = temp['fund_fundmanager']

In [34]:
data = w.wss(df['wind_code'].tolist(), "fund_dq_status","tradeDate=2017-04-13")

In [35]:
temp = wind2df(data)

In [26]:
df['fund_status'] = temp['fund_dq_status']

In [27]:
df.to_excel(fname, index=False)

In [28]:
df.shape

(696, 7)

In [30]:
for x in df['investtype'].unique():
    print x

被动指数型基金
增强指数型基金
普通股票型基金
偏股混合型基金


In [31]:
pnl = pd.read_pickle('C:/Users/jgtzsx01/Documents/workspace/data/FOF/stock.pkl')
fname = u'%s/股票型基金列表.xlsx'%(DATA_DIR)
df = pd.read_excel(fname)

In [32]:
df[df['investtype'] == u'普通股票型基金'].head()

,sec_name,wind_code,investtype,issue_date,netasset,fundmanager,fund_status
4,嘉实研究阿尔法,000082.OF,普通股票型基金,2013-05-02 00:00:00.005,4.786958e+08,赵宇,开放申购|开放赎回
7,大摩品质生活精选,000309.OF,普通股票型基金,2013-09-23 00:00:00.005,1.591271e+09,袁斌,开放申购|开放赎回
14,鹏华环保产业,000409.OF,普通股票型基金,2014-02-10 00:00:00.005,3.706680e+08,郑川江,开放申购|开放赎回
15,景顺长城优质成长,000411.OF,普通股票型基金,2013-11-21 00:00:00.005,5.502587e+07,丁丹,开放申购|开放赎回
16,景顺长城成长之星,000418.OF,普通股票型基金,2013-11-13 00:00:00.005,7.059492e+07,刘晓明,开放申购|开放赎回


In [33]:
ret_df = pnl[df['wind_code']].minor_xs('1-year return')

In [34]:
ret_df.iloc[-20:].std()

000008.OF          NaN
000042.OF     0.013200
000051.OF          NaN
000059.OF     0.031572
000082.OF     0.018704
000176.OF     0.014331
000248.OF          NaN
000309.OF     0.041086
000311.OF     0.016850
000312.OF     0.014633
000313.OF          NaN
000368.OF     0.012153
000373.OF     0.020100
000376.OF          NaN
000409.OF     0.023283
000411.OF     0.021204
000418.OF     0.022027
000457.OF     0.042339
000471.OF     0.021194
000478.OF     0.029526
000513.OF     0.025909
000524.OF     0.025110
000549.OF     0.032429
000577.OF          NaN
000586.OF     0.035617
000592.OF     0.027773
000594.OF     0.022817
000596.OF     0.023611
000613.OF     0.012556
000628.OF     0.025670
                ...   
519677.OF     0.023175
519686.OF          NaN
519706.OF          NaN
519714.OF     0.031617
519931.OF          NaN
519935.OF          NaN
519965.OF     0.047954
519975.OF     0.052696
530010.OF          NaN
530015.OF          NaN
530018.OF     0.016921
540006.OF     0.019126
540007.OF  

In [46]:
df.loc[:, 'max return'] = ret_df.max().values

In [47]:
ret_df = ret_df.rank(axis=1, pct=True)

In [48]:
df.loc[:, 'max percentage'] = ret_df.max().values

In [49]:
df.loc[:, 'max percentage date'] = ret_df.idxmax().values

In [51]:
df = df[df['max percentage'] > 0.9]

In [53]:
df = df.dropna()

In [54]:
df.loc[:, 'max percentage'] = 1 - df.loc[:, 'max percentage']

In [56]:
df = df.sort_values('max return', ascending=False)